# 기본 데이터 수집

In [77]:
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm
import requests

import duckdb

### 종목 기본정보 - kor_info

In [97]:
url = 'https://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',  #핵심 API!
    'mktId': 'ALL',   # STK: 코스피, KSQ: 코스닥, ALL: 전체
    'share': '1',
    'csvxls_isNo': 'false'
}

res = requests.post(url, headers=headers, data=payload)
json_data = res.json()
kor_info_raw = pd.DataFrame(json_data['OutBlock_1'])

# 숫자로 변환
cols_to_convert = ['PARVAL', 'LIST_SHRS']  # 변환할 컬럼 목록
for col in cols_to_convert:
    kor_info_raw[col] = kor_info_raw[col].astype(str).str.replace(',', '', regex=False)
    kor_info_raw[col] = pd.to_numeric(kor_info_raw[col], errors='coerce')
# 날짜형 변환
kor_info_raw['LIST_DD'] = pd.to_datetime(kor_info_raw['LIST_DD'], format='%Y/%m/%d')


kor_info = duckdb.query(
    """
    select
        ISU_CD as 표준티커,
        ISU_SRT_CD as 티커,
        ISU_ABBRV as 종목명,
        MKT_TP_NM as 시장구분,
        LIST_DD AS 상장일,
        KIND_STKCERT_TP_NM as 증권구분,
        PARVAL as 액면가,
    from kor_info_raw
    """
).fetchdf()

kor_info.head()

,표준티커,티커,종목명,시장구분,상장일,증권구분,액면가
0,KR7098120009,098120,마이크로컨텍솔,KOSDAQ,2008-09-23,보통주,500.0
1,KR7009520008,009520,포스코엠텍,KOSDAQ GLOBAL,1997-11-10,보통주,500.0
2,KR7095570008,095570,AJ네트웍스,KOSPI,2015-08-21,보통주,1000.0
3,KR7006840003,006840,AK홀딩스,KOSPI,1999-08-11,보통주,5000.0
4,KR7282330000,282330,BGF리테일,KOSPI,2017-12-08,보통주,1000.0


### 전종목 현재가 - kor_price

In [29]:
# 실시간
today_str = datetime.today().strftime('%Y%m%d')

# 특정 조회날짜
# today_str = 20250715

In [98]:
# 요청 URL
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

# 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

# POST 요청의 form data (payload)
payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',  # 전종목 시세
    'locale': 'ko_KR',
    'mktId': 'ALL',             # 코스피: STK, 코스닥: KSQ, ALL로 하면 전체
    'trdDd': today_str,        # 특정 날짜
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false'
}

res = requests.post(url, headers=headers, data=payload)
data_json = res.json()
kor_price_raw = pd.DataFrame(data_json['OutBlock_1'])

# 숫자로 변환
exclude_cols = ['ISU_SRT_CD', 'ISU_CD', 'ISU_ABBRV', 'MKT_NM', 'SECT_TP_NM', 'MKT_ID'] # 제외할 칼럼 목록
for col in kor_price_raw.select_dtypes(include='object').columns:
    if col not in exclude_cols:
        kor_price_raw[col] = kor_price_raw[col].str.replace(',', '', regex=False)
        kor_price_raw[col] = pd.to_numeric(kor_price_raw[col], errors='coerce')


kor_price = duckdb.query(
    """
    select
        ISU_SRT_CD as 티커,
        TDD_CLSPRC as 종가,
        CMPPREVDD_PRC as 등락,
        FLUC_RT as 등락률,
        TDD_OPNPRC as 시가,
        TDD_HGPRC as 고가,
        TDD_LWPRC as 저가,
        ACC_TRDVOL as 거래량,
        ACC_TRDVAL as 거래대금,
        MKTCAP as 시가총액,
        LIST_SHRS as 상장주식수
    from kor_price_raw
    """
).fetchdf()

kor_price.head(50)

,티커,종가,등락,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,060310,2095,-15,-0.71,2115,2120,2080,285268,598347045,111158688800,53059040
1,095570,4440,-60,-1.33,4500,4500,4365,373737,1645031290,200922249960,45252759
2,006840,12770,40,0.31,12970,12970,12620,7969,101120800,169171353970,13247561
3,054620,4890,-50,-1.01,4895,4980,4855,21958,107275325,97282740690,19894221
4,265520,17900,-510,-2.77,18520,18610,17820,159108,2866219815,273537435900,15281421
5,211270,12640,-10,-0.08,12720,12730,12530,19976,251449630,190640322560,15082304
6,109960,358,13,3.77,350,381,340,1622653,588229766,79042558302,220789269
7,139050,2805,0,0.00,0,0,0,0,0,24247965555,8644551
8,027410,4285,-25,-0.58,4310,4345,4225,194334,827540310,410146449435,95716791
9,282330,121000,700,0.58,120800,121900,118800,61899,7433028506,2091352626000,17283906


### 업종분류현황 - kor_sector
- KONEX는 업종분류 없음

In [99]:
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

# 시장별 결과를 담을 딕셔너리
sector_data = {}

# 시장코드 반복
for mkt_id in ['STK', 'KSQ']:
    payload = {
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT03901',
        'locale': 'ko_KR',
        'mktId': mkt_id,
        'trdDd': today_str,
        'money': '1',
        'csvxls_isNo': 'false'
    }

    res = requests.post(url, headers=headers, data=payload)
    data = res.json()

    # block1 키로부터 DataFrame 생성
    sector_df = pd.DataFrame(data.get('block1', []))
    sector_data[f"{mkt_id}_sector_raw"] = sector_df

STK_sector_raw = sector_data['STK_sector_raw']
KSQ_sector_raw = sector_data['KSQ_sector_raw']


kor_sector = duckdb.query(
    """
    select
        ISU_SRT_CD as 티커,
        IDX_IND_NM as 업종명
    from STK_sector_raw

    UNION ALL

    select
        ISU_SRT_CD as 티커,
        IDX_IND_NM as 업종명
    from KSQ_sector_raw
    """
).fetchdf()

kor_sector.head()

,티커,업종명
0,095570,일반서비스
1,006840,기타금융
2,027410,기타금융
3,282330,유통
4,138930,기타금융


# 본 분석

### 개별일반조회 - unit_info

In [ ]:
# 요청 URL
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

# 요청 헤더
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

# 요청 본문
payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT02101', # 1: 일반정보 3: 개요 4: 재무정보
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_4': '005930/삼성전자', ######
    'isuCd': 'KR7005930003', ######
    'isuCd2': 'KR7005930003', ######
    'codeNmisuCd_finder_stkisu0_4': '삼성전자', ######
    'param1isuCd_finder_stkisu0_4': 'ALL',
    'csvxls_isNo': 'false'
}

# 요청 전송
res = requests.post(url, headers=headers, data=payload)
data = res.json()

# DataFrame으로 변환
df = pd.DataFrame([data])
df.columns = ['표준티커', '종목명', '티커', '시장구분', '현재가', '등락', '증감라벨', '등락률',
              '시가', '고가', '저가', '거래량', '거래대금', '시가총액_백만원', '52주최고', '52주최저',
              '외국인비율', 'PER', 'PBR', '대용가', '배당수익률', '날짜', '날짜_시간']

# 숫자형 변환
for col in ['현재가', '등락', '등락률', '시가', '고가', '저가', '52주최고', '52주최저',
            'PER', 'PBR', '거래량', '거래대금', '외국인비율' ,'대용가' ,'배당수익률']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')


# 날짜형 변환
df['날짜_시간'] = pd.to_datetime(df['날짜_시간'], format='%Y.%m.%d %p %I:%M:%S')


unit_info = df[['날짜_시간', '표준티커', '종목명', '현재가', '등락', '등락률',
         '시가', '고가', '저가', '52주최고', '52주최저',
         'PER', 'PBR', '거래량', '거래대금', '외국인비율', '배당수익률', '대용가']]
unit_info.head()



,날짜_시간,표준티커,종목명,현재가,등락,등락률,시가,고가,저가,52주최고,52주최저,PER,PBR,거래량,거래대금,외국인비율,배당수익률,대용가
0,2025-07-19 00:08:24,KR7005930003,삼성전자,67100.0,400.0,0.6,67000.0,67800.0,66300.0,86900.0,49900.0,13.56,1.16,23951531.0,1.607582e+12,50.08,2.15,50690.0


#### 일반조회 종목열거 코드

In [95]:
# 검색종목 칼럼 + 검색조건
df = duckdb.query(
    """
    select *
    from kor_info
    where 시장구분 = 'KOSDAQ'
    """
).fetchdf()
kor_info_list = df.to_dict('records')

# 결과 저장용 리스트
results = []

# 반복 처리
for t in tqdm(kor_info_list):
    try:
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

        headers = {
            'User-Agent': 'Mozilla/5.0',
            'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
        }

        payload = {
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT02101',
            'locale': 'ko_KR',
            'tboxisuCd_finder_stkisu0_4': f"{t['티커']}/{t['종목명']}",
            'isuCd': t['표준티커'],
            'isuCd2': t['표준티커'],
            'codeNmisuCd_finder_stkisu0_4': t['종목명'],
            'param1isuCd_finder_stkisu0_4': 'ALL',
            'csvxls_isNo': 'false'
        }

        res = requests.post(url, headers=headers, data=payload)
        data = res.json()

        # JSON 응답에서 DataFrame 생성
        df = pd.DataFrame([data])
        df.columns = ['표준티커', '종목명', '티커', '시장구분', '현재가', '등락', '증감라벨', '등락률',
                      '시가', '고가', '저가', '거래량', '거래대금', '시가총액_백만원', '52주최고', '52주최저',
                      '외국인비율', 'PER', 'PBR', '대용가', '배당수익률', '날짜', '날짜_시간']

        # 숫자형 변환
        for col in ['현재가', '등락', '등락률', '시가', '고가', '저가', '52주최고', '52주최저',
                    'PER', 'PBR', '거래량', '거래대금', '외국인비율', '대용가', '배당수익률']:
            df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')

        # 날짜형 변환
        df['날짜_시간'] = pd.to_datetime(df['날짜_시간'], format='%Y.%m.%d %p %I:%M:%S')

        # 원하는 컬럼만 추출
        unit_info = df[['날짜_시간', '시장구분', '티커', '종목명',
                        '시가', '고가', '저가', '52주최고', '52주최저', 'PER', 'PBR', '외국인비율', '배당수익률', '대용가']]

        results.append(unit_info)

        # 요청 간 0.5~1.0초 랜덤 대기 (너무 빠르면 차단될 수 있음)
        time.sleep(0.5)

    except Exception as e:
        print(f"{t['종목명']} 실패: {e}")
        continue

# 결과 통합
all_info = pd.concat(results, ignore_index=True)
all_info


100%|██████████| 1743/1743 [17:13<00:00,  1.69it/s]


,날짜_시간,시장구분,티커,종목명,시가,고가,저가,52주최고,52주최저,PER,PBR,외국인비율,배당수익률,대용가
0,2025-07-19 02:09:10,KOSDAQ,098120,마이크로컨텍솔,19950,20100,18210,22700,4250,15.42,2.53,13.57,0.40,12210
1,2025-07-19 02:09:11,KOSDAQ,058820,CMG제약,2080,2120,2065,2975,1600,110.53,1.54,4.18,0.00,1550
2,2025-07-19 02:09:12,KOSDAQ,050120,ES큐브,2790,2795,2725,3010,1574,40.44,0.43,3.09,0.00,1720
3,2025-07-19 02:09:12,KOSDAQ,024850,HLB이노베이션,1885,1889,1832,3990,1802,NaN,1.09,2.32,0.00,1350
4,2025-07-19 02:09:13,KOSDAQ,024120,KB오토시스,3680,3735,3600,4250,2875,14.20,0.35,5.96,5.50,2730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,2025-07-19 02:26:21,KOSDAQ,024060,흥구석유,13040,13240,12920,21800,9300,304.19,2.41,1.39,0.23,7970
1739,2025-07-19 02:26:22,KOSDAQ,010240,흥국,5410,5410,5270,5680,4570,5.59,0.54,4.79,4.14,3960
1740,2025-07-19 02:26:22,KOSDAQ,189980,흥국에프엔비,1972,1981,1941,2255,1435,14.85,0.81,0.83,3.54,1380
1741,2025-07-19 02:26:23,KOSDAQ,037440,희림,5010,5030,4975,7150,4400,4.90,0.77,7.20,2.99,3500


In [96]:
all_info_order = duckdb.query(
    """
    select *
    from all_info
    order by PER desc
    """
).fetchdf()
all_info_order

,날짜_시간,시장구분,티커,종목명,시가,고가,저가,52주최고,52주최저,PER,PBR,외국인비율,배당수익률,대용가
0,2025-07-19 02:20:37,KOSDAQ,054210,이랜텍,7100,7130,6930,8350,4055,3555.00,0.71,4.05,1.13,4940
1,2025-07-19 02:11:59,KOSDAQ,277810,레인보우로보틱스,275000,275000,269000,417500,113500,2490.91,40.04,4.18,0.00,202020
2,2025-07-19 02:24:43,KOSDAQ,370090,퓨런티어,15100,15290,14820,29250,12860,1873.75,2.70,3.97,0.00,9270
3,2025-07-19 02:19:02,KOSDAQ,039200,오스코텍,34800,34800,34000,43450,21700,1495.65,10.30,9.57,0.00,24940
4,2025-07-19 02:14:59,KOSDAQ,408900,스튜디오미르,4695,4730,4380,5130,2260,1466.67,3.67,0.49,0.00,2910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,2025-07-19 02:26:14,KOSDAQ,090710,휴림로봇,3085,3200,3045,3495,1074,NaN,2.73,1.63,0.00,2150
1739,2025-07-19 02:26:16,KOSDAQ,205470,휴마시스,1461,1461,1435,3005,1254,NaN,0.71,6.15,0.00,900
1740,2025-07-19 02:26:16,KOSDAQ,115160,휴맥스,1046,1060,1041,2035,990,NaN,0.29,4.45,0.00,710
1741,2025-07-19 02:26:17,KOSDAQ,028080,휴맥스홀딩스,2160,2175,2125,4010,1705,NaN,0.44,1.35,0.00,1280


### 개별재무정보 - unit_financial

In [105]:
# 요청 URL
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

# 요청 헤더
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
}

# 요청 본문
payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT02104', # 1: 일반정보 3: 개요 4: 재무정보
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_4': '005930/삼성전자', ######
    'isuCd': 'KR7005930003', ######
    'isuCd2': 'KR7005930003', ######
    'codeNmisuCd_finder_stkisu0_4': '삼성전자', ######
    'param1isuCd_finder_stkisu0_4': 'ALL',
    'csvxls_isNo': 'false'
}

# 요청 전송
res = requests.post(url, headers=headers, data=payload)
data = res.json()

# DataFrame으로 변환
df = pd.DataFrame([data])
df['표준티커'] =  'KR7005930003' ######
df.columns = ['CONN_TP', '자산', '부채', '자본금', '자본', '매출액', '영업이익', '당기순이익', '날짜_시간', '표준티커']

# 숫자형 변환
for col in ['자산', '부채', '자본', '자본금', '매출액', '영업이익', '당기순이익']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')


# 날짜형 변환
df['날짜_시간'] = pd.to_datetime(df['날짜_시간'], format='%Y.%m.%d %p %I:%M:%S')

unit_financial = df[['날짜_시간', '표준티커', '자산', '부채', '자본', '자본금', '매출액', '영업이익', '당기순이익']]
unit_financial.head()


,날짜_시간,표준티커,자산,부채,자본,자본금,매출액,영업이익,당기순이익
0,2025-07-19 02:44:47,KR7005930003,514531948000000,112339878000000,402192070000000,897514000000,300870903000000,32725961000000,34451351000000


#### 재무정보 종목열거 코드

In [91]:
# 검색 설정
df = duckdb.query(
    """
    select *
    from kor_info
    where 시장구분 = 'KOSDAQ'
    """
).fetchdf()

kor_info_list = df.to_dict('records')

# 결과 저장용 리스트
results = []

# 반복 처리
for t in tqdm(kor_info_list):
    try:
        url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

        headers = {
            'User-Agent': 'Mozilla/5.0',
            'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201'
        }

        payload = {
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT02104',
            'locale': 'ko_KR',
            'tboxisuCd_finder_stkisu0_4': f"{t['티커']}/{t['종목명']}",
            'isuCd': t['표준티커'],
            'isuCd2': t['표준티커'],
            'codeNmisuCd_finder_stkisu0_4': t['종목명'],
            'param1isuCd_finder_stkisu0_4': 'ALL',
            'csvxls_isNo': 'false'
        }

        res = requests.post(url, headers=headers, data=payload)
        data = res.json()

        # 응답을 DataFrame으로 변환
        df = pd.DataFrame([data])
        df['표준티커'] = t['표준티커']

        df.columns = ['CONN_TP', '자산', '부채', '자본금', '자본', '매출액', '영업이익', '당기순이익', '날짜_시간', '표준티커']

        # 숫자형 변환 (결측치는 NaN으로)
        for col in ['자산', '부채', '자본', '자본금', '매출액', '영업이익', '당기순이익']:
            df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')

        # 날짜형 변환
        df['날짜_시간'] = pd.to_datetime(df['날짜_시간'], format='%Y.%m.%d %p %I:%M:%S')

        # 원하는 컬럼만 추출
        unit_financial = df[['날짜_시간', '표준티커', '자산', '부채', '자본', '자본금', '매출액', '영업이익', '당기순이익']]

        results.append(unit_financial)

        # 과도한 요청 방지
        time.sleep(0.7)

    except Exception as e:
        print(f"### {t['종목명']} 오류 발생: {e}")
        continue

# ▶ 결과 합치기
all_financial = pd.concat(results, ignore_index=True)
all_financial

100%|██████████| 1743/1743 [21:58<00:00,  1.32it/s]


,날짜_시간,표준티커,자산,부채,자본,자본금,매출액,영업이익,당기순이익
0,2025-07-19 01:34:59,KR7098120009,76917404000,11858904000,65058500000,4156383000,69664083000,10204007000,10680949000
1,2025-07-19 01:35:00,KR7058820002,303436817000,113894698000,189542119000,69446122000,99099733000,1020329000,2694712000
2,2025-07-19 01:35:01,KR7050120005,90410906000,4509175000,85901731000,69092016000,16172076000,-3743000000,909489000
3,2025-07-19 01:35:01,KR7024850000,268011067000,16028155000,251982911000,72152062000,25355832000,-11882543000,-12369788000
4,2025-07-19 01:35:02,KR7024120008,240702310000,121719437000,118982873000,5750000000,210230561000,4634465000,2944075000
...,...,...,...,...,...,...,...,...,...
1738,2025-07-19 01:56:54,KR7024060006,92926944000,14004200000,78922744000,1500000000,116852401000,-282773000,620411000
1739,2025-07-19 01:56:55,KR7010240000,126303882000,17120948000,109182934000,6161348000,128710756000,8787501000,10550352000
1740,2025-07-19 01:56:56,KR7189980006,221083288000,123960969000,97122320000,4013783000,102624065000,10742389000,5197335000
1741,2025-07-19 01:56:56,KR7037440005,208298374000,126168802000,82129572000,6961238000,240961112000,15279029000,13428368000


In [92]:
all_financial_order = duckdb.query(
    """
    select *
    from all_financial
    order by 자산 desc
    """
).fetchdf()
all_financial_order

,날짜_시간,표준티커,자산,부채,자본,자본금,매출액,영업이익,당기순이익
0,2025-07-19 01:36:41,KR7032190001,57757101614000,51286171478000,6470930136000,19150000000,12136122777000,1205869585000,865904046000
1,2025-07-19 01:46:55,KR7078020005,8779672481000,7908255209000,871417273000,363969375000,1653673426000,21808378000,16619943000
2,2025-07-19 01:52:28,KR7151860004,6213358821000,3254441403000,2958917418000,26853361000,7284861547000,226534825000,214231564000
3,2025-07-19 01:41:04,KR7038540001,5104241131000,4533973404000,570267727000,57187237000,870250386000,-214829325000,-215053660000
4,2025-07-19 01:41:36,KR7015750003,4563110592000,2721044802000,1842065789000,40000000000,4245092708000,205882638000,135061913000
...,...,...,...,...,...,...,...,...,...
1738,2025-07-19 01:55:29,KR7475240008,7187722000,928424000,6259298000,320000000,0,-37569000,88636000
1739,2025-07-19 01:56:22,KR7498390004,2490592000,2002212000,488380000,21000000,0,-9408000,-15316000
1740,2025-07-19 01:49:09,KR7493790000,2280313000,1738508000,541805000,31000000,0,-17101000,-25711000
1741,2025-07-19 01:42:50,KR7496070004,1981968000,1200947000,781020000,62000000,0,-18715000,-18882000


### 개별종목시세 - unit_price

In [106]:
# 요청 URL
url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

# 요청 헤더
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest"
}

# 요청 파라미터 
data = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT01701",
    "locale": "ko_KR",
    "tboxisuCd_finder_stkisu0_1": "035720/카카오", ######
    "isuCd": "KR7035720002", ######
    "isuCd2": "KR7035720002", ######
    "codeNmisuCd_finder_stkisu0_1": "카카오", ######
    "param1isuCd_finder_stkisu0_1": "ALL",
    "strtDd": "20230719", ######
    "endDd": "20250718", ######
    "adjStkPrc_check": "Y",
    "adjStkPrc": "2",
    "share": "1",
    "money": "1",
    "csvxls_isNo": "false"
}

# POST 요청
res = requests.post(url, headers=headers, data=data)

# 결과
json_data = res.json()
df = pd.DataFrame(json_data['output'])

# 컬럼 이름 정리
df.columns = [col.strip() for col in df.columns]
df = df.rename(columns={
    'TRD_DD': '일자',
    'TDD_CLSPRC': '종가',
    'FLUC_TP_CD': '등락구분',
    'CMPPREVDD_PRC': '등락',
    'FLUC_RT': '등락률',
    'TDD_OPNPRC': '시가',
    'TDD_HGPRC': '고가',
    'TDD_LWPRC': '저가',
    'ACC_TRDVOL': '거래량',
    'ACC_TRDVAL': '거래대금',
    'MKTCAP' : '시가총액',
    'LIST_SHRS' : '상장주식수'
})

# 숫자형 변환
for col in ['종가', '등락', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')

df['일자'] = pd.to_datetime(df['일자'], format='%Y/%m/%d')
df['표준티커'] = "KR7035720002" ######

unit_price = df[['일자', '표준티커', '종가', '등락', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']]
unit_price

,일자,표준티커,종가,등락,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2025-07-18,KR7035720002,56900,-600,57700,57800,56100,1334076,75746376350,25150580781800,442013722
1,2025-07-17,KR7035720002,57500,1400,57500,59100,57000,3318001,191514032850,25415789015000,442013722
2,2025-07-16,KR7035720002,56100,-1100,56900,57600,56000,1844953,104264641350,24796969804200,442013722
3,2025-07-15,KR7035720002,57200,-1100,57400,58000,56300,2979693,169395188650,25283184898400,442013722
4,2025-07-14,KR7035720002,58300,-1700,59500,59600,56500,3469359,200495860900,25769399992600,442013722
...,...,...,...,...,...,...,...,...,...,...,...
481,2023-07-25,KR7035720002,48850,-1000,49600,49800,48800,1788062,87682627150,21702261716500,444263290
482,2023-07-24,KR7035720002,49850,-750,50100,50300,49400,1943708,96571041850,22146525006500,444263290
483,2023-07-21,KR7035720002,50600,100,50000,50800,49750,1203404,60651477550,22479722474000,444263290
484,2023-07-20,KR7035720002,50500,100,50100,51100,50100,1150775,58316036900,22435296145000,444263290


#### 5영업일 unit수익률 코드

In [ ]:
# 요청 URL
url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

# 요청 헤더
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest"
}

# 요청 파라미터
data = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT01701",
    "locale": "ko_KR",
    "tboxisuCd_finder_stkisu0_1": "403550/쏘카", ######
    "isuCd": "KR7403550007", ######
    "isuCd2": "KR7403550007", ######
    "codeNmisuCd_finder_stkisu0_1": "쏘카", ######
    "param1isuCd_finder_stkisu0_1": "ALL",
    "strtDd": "20250714", ######
    "endDd": "20250718", ######
    "adjStkPrc_check": "Y",
    "adjStkPrc": "2",
    "share": "1",
    "money": "1",
    "csvxls_isNo": "false"
}

res = requests.post(url, headers=headers, data=data)

json_data = res.json()
df = pd.DataFrame(json_data['output'])

# 컬럼 이름 정리
df.columns = [col.strip() for col in df.columns]
df = df.rename(columns={
    'TRD_DD': '일자',
    'TDD_CLSPRC': '종가',
    'FLUC_TP_CD': '등락구분',
    'CMPPREVDD_PRC': '등락',
    'FLUC_RT': '등락률',
    'TDD_OPNPRC': '시가',
    'TDD_HGPRC': '고가',
    'TDD_LWPRC': '저가',
    'ACC_TRDVOL': '거래량',
    'ACC_TRDVAL': '거래대금',
    'MKTCAP' : '시가총액',
    'LIST_SHRS' : '상장주식수'
})

# 숫자형 변환
for col in ['종가', '등락', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')

df['일자'] = pd.to_datetime(df['일자'], format='%Y/%m/%d')
df['표준티커'] = "KR7005930003" ######

unit_price = df[['일자', '표준티커', '종가', '등락', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']]

#################
return_5day = duckdb.query(
    """
    select 일자, 표준티커, 종가,
        round((종가 - lag(종가, 4) over (order by 일자))
        / lag(종가, 4) over (order by 일자) * 100, 2) as 수익률_5일
    from unit_price
    order by 일자 desc
    limit 1
    """
).fetchdf()

return_5day

,일자,표준티커,종가,수익률_5일
0,2025-07-18,KR7005930003,13220.0,0.69


#### 5영업일 종목열거 코드

In [103]:
# 종목 리스트(표준티커, 티커, 종목명) + 조건설정(시장구분이나 섹터 특정 등)
df = duckdb.query(
    """
    select *
    from kor_info
    where 시장구분 = 'KOSDAQ'
    """
).fetchdf()

kor_info_list = df.to_dict('records')

# 결과 저장용 리스트
results = []

# 반복 처리
for t in tqdm(kor_info_list):
    # 요청 파라미터
    data = {
        "bld": "dbms/MDC/STAT/standard/MDCSTAT01701",
        "locale": "ko_KR",
        "tboxisuCd_finder_stkisu0_1": f"{t['티커']}/{t['종목명']}",
        "isuCd": t["표준티커"],
        "isuCd2": t["표준티커"],
        "codeNmisuCd_finder_stkisu0_1": t["종목명"],
        "param1isuCd_finder_stkisu0_1": "ALL",
        "strtDd": "20250714", #########
        "endDd": "20250718", #########
        "adjStkPrc_check": "Y",
        "adjStkPrc": "2",
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false"
    }

    # 요청
    url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Requested-With": "XMLHttpRequest"
    }
    res = requests.post(url, headers=headers, data=data)
    time.sleep(0.05)

    try:
        json_data = res.json()
        df = pd.DataFrame(json_data['output'])

        # 컬럼 이름 정리 및 숫자형 변환
        df.columns = [col.strip() for col in df.columns]
        df = df.rename(columns={
            'TRD_DD': '일자',
            'TDD_CLSPRC': '종가',
            'FLUC_TP_CD': '등락구분',
            'CMPPREVDD_PRC': '등락',
            'FLUC_RT': '등락률',
            'TDD_OPNPRC': '시가',
            'TDD_HGPRC': '고가',
            'TDD_LWPRC': '저가',
            'ACC_TRDVOL': '거래량',
            'ACC_TRDVAL': '거래대금',
            'MKTCAP': '시가총액',
            'LIST_SHRS': '상장주식수'
        })
        for col in ['종가', '등락', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수']:
            df[col] = pd.to_numeric(df[col].str.replace(",", ""), errors='coerce')

        df['일자'] = pd.to_datetime(df['일자'], format='%Y/%m/%d')
        df['표준티커'] = t["표준티커"]

        unit_price = df[['일자', '표준티커', '종가']]

        # 5일 수익률 계산 ###############
        return_5day = duckdb.query(
            """
            select 일자, 표준티커, 종가,
                round((종가 - lag(종가, 4) over (order by 일자))
                / lag(종가, 4) over (order by 일자) * 100, 2) as 수익률_5일
            from unit_price
            order by 일자 desc
            limit 1
            """
        ).fetchdf()

        results.append(return_5day)

    except Exception as e:
        print(f"[ERROR] {t['종목명']} 처리 중 오류 발생: {e}")

# 모든 결과 하나로 합치기
return_5day = pd.concat(results, ignore_index=True)

# 결과 출력
return_5day.head()

100%|██████████| 1743/1743 [03:11<00:00,  9.11it/s]


,일자,표준티커,종가,수익률_5일
0,2025-07-18,KR7098120009,19820,-6.51
1,2025-07-18,KR7058820002,2100,4.74
2,2025-07-18,KR7050120005,2750,-0.54
3,2025-07-18,KR7024850000,1851,-2.27
4,2025-07-18,KR7024120008,3635,0.14


In [104]:
sector_5day_return_rankone = duckdb.query(
    """
    select 일자, 표준티커, 종가, 수익률_5일
    from return_5day
    order by 수익률_5일 desc
    """
).fetchdf()

sector_5day_return_rankone

,일자,표준티커,종가,수익률_5일
0,2025-07-18,KR7049470008,2000,119.54
1,2025-07-18,KR7441270006,11350,58.96
2,2025-07-18,KR7087010005,294500,53.23
3,2025-07-18,KR7362320004,9350,50.81
4,2025-07-18,KR7308080001,4000,36.75
...,...,...,...,...
1738,2025-07-18,KR7083660001,1847,-20.39
1739,2025-07-18,KR7036630002,5250,-21.29
1740,2025-07-18,KR7462310004,10770,-23.02
1741,2025-07-18,KR7276040003,2345,-26.95


In [60]:
sector_5day_return_rankone = duckdb.query(
    """
    select *
    from (
        select 
            일자, 시장구분, i.표준티커, i.티커, 업종명, 종목명, 수익률_5일,
            rank() over(partition by 업종명 order by 수익률_5일 desc) as sector_rank
        from return_5day r
        join kor_info i using(표준티커)
        join kor_sector s on i.티커 = s.티커
    ) t
    where sector_rank = 1
    order by 수익률_5일 desc
    """
).fetchdf()

sector_5day_return_rankone

,일자,시장구분,표준티커,티커,업종명,종목명,수익률_5일,sector_rank
0,2025-07-18,KOSPI,KR7103140000,103140,금속,풍산,21.45,1
1,2025-07-18,KOSPI,KR7000100008,000100,제약,유한양행,18.49,1
2,2025-07-18,KOSPI,KR7014580005,014580,비금속,태경비케이,15.00,1
3,2025-07-18,KOSPI,KR7004140000,004140,운송·창고,동방,14.15,1
4,2025-07-18,KOSPI,KR7003670007,003670,전기·전자,포스코퓨처엠,14.03,1
5,2025-07-18,KOSPI,KR7178920005,178920,화학,PI첨단소재,13.83,1
6,2025-07-18,KOSPI,KR7010620003,010620,운송장비·부품,HD현대미포,12.80,1
7,2025-07-18,KOSPI,KR7229640008,229640,기타금융,LS에코에너지,12.76,1
8,2025-07-18,KOSPI,KR7484870001,484870,기계·장비,엠앤씨솔루션,10.75,1
9,2025-07-18,KOSPI,KR7016880007,016880,일반서비스,웅진,9.77,1


# SQL

In [10]:
df = duckdb.query(
    """
    select *
    from kor_info
    where 시장구분 = 'KOSPI'
    """
).fetchdf()

,count_star()
0,962


In [ ]:
duckdb.query(
    """
    select count(*)
    from kor_price
    -- group by 시장구분
    """
).fetchdf()

# DART 전자공시

In [ ]:
import requests
import zipfile
import io
import pandas as pd

api_key = '3adc2cc74bd0f5806d6364d34476a332aa420bd5'

# DART에서 전체 기업 코드 ZIP 다운로드
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'
res = requests.get(url)
with zipfile.ZipFile(io.BytesIO(res.content)) as z:
    xml_content = z.read(z.namelist()[0])

# XML → DataFrame 변환
import xml.etree.ElementTree as ET
tree = ET.fromstring(xml_content)
corp_list = []
for child in tree:
    corp = {item.tag: item.text for item in child}
    corp_list.append(corp)
corp_df = pd.DataFrame(corp_list)

# 예: 삼성전자 찾기
corp_code = corp_df[corp_df['corp_name'] == '삼성전자']['corp_code'].values[0]

In [ ]:
# 사업보고서 (2023년 기준)
url = f'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
    'bsns_year': '2024',
    'reprt_code': '11011',  # 11011: 사업보고서, 11012: 1Q, 11013: 반기, 11014: 3Q
    'fs_div': 'CFS'         # 연결: CFS / 개별: OFS
}

res = requests.get(url, params=params)
data = res.json()

df_fnl = pd.DataFrame(data['list'])
df_fnl